# Data Preparation for stock data  
### Data 1: Stock Price & Volume difference from start of year to end of year  
### Data 2: Stock fundamental indicators - Balance sheet, income statement  

In [1]:
import sqlite3 as lite
import sys
import pandas as pd

### Table schema

In [2]:
con = lite.connect('test.db')

In [7]:
# CREATE TABLE stock_master (ID PRIMARY KEY, symbol TEXT, descript TEXT, exchange TEXT);
query = "select * from stock_master"
df = pd.read_sql_query(query, con) 
df.head(1)

,ID,symbol,descript,exchange
0,21480311,A,ARMOR MINERALS INC.,TOR-V


In [8]:
# CREATE TABLE IF NOT EXISTS "master_ext" ( "index" INTEGER, "ID" INTEGER, "symbol" TEXT, "descript" TEXT, 
# "exchange" TEXT, "ticket" TEXT, "industry" TEXT, "sector" TEXT, "json" TEXT);
query = "select * from master_ext"
df = pd.read_sql_query(query, con) 
df.head(1)

,index,ID,symbol,descript,exchange,ticket,industry,sector,json
0,0,21480311,A,ARMOR MINERALS INC.,TOR-V,A.V,Other Industrial Metals & Mining,Basic Materials,"b'{""quoteSummary"":{""result"":[{""assetProfile"":{..."


In [9]:
# CREATE TABLE FINANCIAL_ITEMS(SYMBOL TEXT, ID INT, RPT_TYPE TEXT, RPT_PERIOD TEXT, RPT_DATE DATE, 
# ITEM_NAME TEXT, ITEM_VALUE DOUBLE);
query = "select * from financial_items limit 10"
df = pd.read_sql_query(query, con) 
df.head(1)

,SYMBOL,ID,RPT_TYPE,RPT_PERIOD,RPT_DATE,ITEM_NAME,ITEM_VALUE
0,AC,96768,income,annual,2011-12-30,Sales,0.0


### Get the price and volume history

In [75]:
# Difference between end of year to start of year
import yfinance as yf
#https://pypi.org/project/yfinance/
def getTicketHistoricalPrice(ticket, symbol, id):
    df_out = pd.DataFrame()
    try:
        tick = yf.Ticker(ticket)
        hist = tick.history(interval = "3mo", start="2012-01-01", end="2017-01-01").dropna()
        df = hist.reset_index()[['Date','Close','Volume']]    
        df['yr'] = df.Date.dt.year
        df_out = (df.groupby(df.yr).last() - df.groupby(df.yr).first())/df.groupby(df.yr).mean()
        df_out = df_out.reset_index().drop(columns = ['Date'])
        df_out['symbol'] = symbol
        df_out['id'] = id
        df_out = df_out[['yr','id','symbol','Close','Volume']]
    except:
        pass
    return df_out
df = getTicketHistoricalPrice('A.V', 'A', 21480311)[['yr','id','symbol','Close','Volume']]
df

,yr,id,symbol,Close,Volume
0,2012,21480311,A,-1.142857,0.471281
1,2013,21480311,A,-1.894737,-0.894410
2,2014,21480311,A,-1.391304,-2.453333
3,2015,21480311,A,0.210526,0.153545
4,2016,21480311,A,0.568047,-0.006796


In [ ]:
query = "select ID, symbol, ticket from master_ext"
df = pd.read_sql_query(query, con).drop_duplicates() 
df_out = pd.DataFrame()

for ind,content in df.iterrows():
    df_out = df_out.append(getTicketHistoricalPrice(content[2], content[1], content[0]))    

In [82]:
print(df_out.head(3))
print(df_out.tail(3))
print(len(set(df_out.id)))

     yr        id symbol     Close    Volume
0  2012  21480311      A -1.142857  0.471281
1  2013  21480311      A -1.894737 -0.894410
2  2014  21480311      A -1.391304 -2.453333
     yr     id symbol     Close    Volume
2  2014  34084    NOT -0.533333 -0.993464
3  2015  34084    NOT -0.421053 -1.398679
4  2016  34084    NOT -0.241379 -0.399013
1414


In [83]:
df_out.to_csv("sample_price_volume.csv")

### Get fundamental 

In [84]:
query = "select id, symbol, strftime('%Y', rpt_date) as yr, item_name, item_value from financial_items where RPT_PERIOD = 'annual'  "
df = pd.read_sql_query(query, con)
df.head(1)

,ID,SYMBOL,yr,ITEM_NAME,ITEM_VALUE
0,96768,AC,2011,Sales,0.0


In [85]:
set(df.yr)

{'2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017'}

In [86]:
df.yr = pd.to_numeric(df.yr)
df = df[(df.yr > 2011) & (df.yr < 2017)]
df_out = df.pivot_table(index=['ID','SYMBOL', 'yr'], values=['ITEM_VALUE'], columns=['ITEM_NAME']).reset_index()
df_out.columns = [' '.join(col).strip() for col in df_out.columns.values]
df_out.head(10)

,ID,SYMBOL,yr,ITEM_VALUE (Decrease) Increase In Other Current Liabilities,ITEM_VALUE (Decrease) Increase In Payables,ITEM_VALUE (Increase) Decrease In Inventories,ITEM_VALUE (Increase) Decrease In Other Current Assets,ITEM_VALUE (Increase) Decrease In Other Working Capital,ITEM_VALUE (Increase) Decrease In Receivables,ITEM_VALUE Accounts Payable,...,ITEM_VALUE Total Current Assets,ITEM_VALUE Total Current Liabilities,ITEM_VALUE Total Equity,ITEM_VALUE Total Income Before Interest Expenses (EBIT),ITEM_VALUE Total Liabilities & Stock Equity,ITEM_VALUE Total Net Income,ITEM_VALUE Total Non-Current Assets,ITEM_VALUE Total Non-Current Liabilities,ITEM_VALUE Treasury Shares,ITEM_VALUE Treasury Stock
0,31228,AIS.H,2012,0.0,0.0,0.0,0.0,0.0,0.0,22140.0,...,475240.0,22140.0,536010.0,0.0,0.0,-94120.0,82920.0,0.0,0.0,0.0
1,31228,AIS.H,2013,0.0,0.0,0.0,0.0,0.0,0.0,30840.0,...,435560.0,30840.0,479480.0,0.0,0.0,-56530.0,74760.0,0.0,0.0,0.0
2,31228,AIS.H,2014,0.0,0.0,0.0,0.0,0.0,0.0,93810.0,...,63040.0,93810.0,93110.0,0.0,0.0,-1010000.0,123890.0,0.0,0.0,0.0
3,31228,AIS.H,2015,0.0,0.0,0.0,0.0,0.0,0.0,345580.0,...,14710.0,346200.0,221750.0,0.0,0.0,-593450.0,553230.0,0.0,0.0,0.0
4,31228,AIS.H,2016,0.0,0.0,0.0,0.0,0.0,0.0,178940.0,...,401780.0,178940.0,1240000.0,0.0,0.0,-757220.0,1020000.0,0.0,0.0,0.0
5,31243,DRX,2012,0.0,0.0,0.0,0.0,0.0,0.0,5550000.0,...,53470000.0,10850000.0,93760000.0,0.0,0.0,1810000.0,55050000.0,3910000.0,0.0,0.0
6,31243,DRX,2013,0.0,0.0,0.0,0.0,0.0,0.0,2250000.0,...,48990000.0,13950000.0,91430000.0,0.0,0.0,-1550000.0,57550000.0,1160000.0,0.0,0.0
7,31243,DRX,2014,0.0,0.0,0.0,0.0,0.0,0.0,4940000.0,...,48960000.0,19340000.0,101830000.0,0.0,0.0,7680000.0,79030000.0,6810000.0,0.0,0.0
8,31243,DRX,2015,0.0,0.0,0.0,0.0,0.0,0.0,15970000.0,...,41500000.0,22020000.0,103960000.0,0.0,0.0,-1570000.0,96320000.0,11840000.0,0.0,0.0
9,31243,DRX,2016,0.0,0.0,0.0,0.0,0.0,0.0,17770000.0,...,42810000.0,21850000.0,107530000.0,0.0,0.0,1700000.0,103670000.0,17090000.0,0.0,0.0


In [87]:
df_out.to_csv("sample.csv")

In [89]:
df_in=pd.read_csv("sample_price_volume.csv")

In [92]:
# overlapping of price data and fundamental data
len(set(df_in.id).intersection(df_out.ID))

1350